## Feature Engineering

* Need to excude the target column from the feature store to avoid data leakage.
* Define a primary key for the feature table.

## Load Data
* Typically we would load from Silver table. 

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import * 
from pyspark.ml.feature import StringIndexer, Word2Vec, VectorAssembler
from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.sql.types import ArrayType, FloatType
import pyspark.sql.functions as F

In [0]:
telco_path = "sdd_dev.sohag_test.telco_customer_churn"
telco_df = spark.read.table(telco_path)
telco_df = telco_df.drop("Churn")
display(telco_df)

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient
fe = FeatureEngineeringClient()

## Create Feature Table
We can create the Feature table using the create_table method.
This method takes a few parameters as inputs:
* name: Name of the feature table as catalog.schema.table_name.
* primary_keys: List of primary keys for the feature table.
* timestamp_col: Timestamp column for the feature table.
* df: Data to insert into the feature table. The schema of feature_df will be used as the feature table schema.
* schema: Schema of the feature table. Note that either schema or feature_df must be provided.
* description: Description of the feature table.
* partition_cols: List of partition columns for the feature table.
* tags: Tags to tag the feature table.

In [0]:
# create a feature table from the dataset
table_name = "sdd_dev.sohag_test.telco_customer_features"

fe.create_table(
    name=table_name,
    primary_keys=["customerID"],
    df=telco_df,
    description="Telco Customer Features",
    tags={"source": "telco_customer_churn"}
)

In [0]:
print("Hello")

## Load Feature Table
We can also look at the metadata of the feature table using the get_table method.

In [0]:
ft = fe.get_table(name = table_name)
print(f"Feature table description: {ft.description}")
print(ft.features)

In [0]:
display(fe.read_table(name=table_name))

## Update Feature Table
### Add a new feature

In [0]:
from pyspark.sql.functions import when
telco_df_updated = telco_df.withColumn("tenure_group",
    when((telco_df.tenure >= 0) & (telco_df.tenure <= 25), "short")
    .when((telco_df.tenure > 25) & (telco_df.tenure <= 50), "mid")
    .when((telco_df.tenure > 50) & (telco_df.tenure <= 75), "long")
    .otherwise("invalid")
)

Select relevant columns. Use appropiate mode (eg: merge) and display the written table for validation

In [0]:
fe.write_table(
    name=table_name, 
    df=telco_df_updated.select("customerID", "tenure_group"),
    mode = "merge")

## Delete existing feature
To remove a feature column from the table you can just drop the column. Let's drop the tenure column. 

In [0]:
%sql
ALTER TABLE sdd_dev.sohag_test.telco_customer_features 
SET TBLPROPERTIES (
    'delta.columnMapping.mode' = 'name',
    'delta.minReaderVersion' = '2', 
    'delta.minWriterVersion' = '5'
);

ALTER TABLE sdd_dev.sohag_test.telco_customer_features 
DROP COLUMNS(tenure);

## Read feature table by version

In [0]:
timestamp_v3 = spark.sql(f"DESCRIBE HISTORY {table_name}").orderBy("version").collect()[2].timestamp
print(timestamp_v3)

In [0]:
display(spark.sql(f"DESCRIBE HISTORY {table_name}").orderBy("version"))

In [0]:
# Use a valid timestamp that is before or at the latest available timestamp
valid_timestamp_v3 = "2025-07-11 19:54:56.0"
telco_df_v3 = (spark.read.option("timestampAsOf", valid_timestamp_v3).table(telco_path))
display(telco_df_v3)

In [0]:
# Display old version of feature table 
feature_df = fe.read_table(
    name=table_name,
    as_of_delta_timestamp = timestamp_v3)

feature_df.printSchema()